In [29]:
import json, time
import pandas as pd
import gzip
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

In [4]:
df.head(60)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,"02 12, 2011"
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,"01 19, 2013"
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,"01 4, 2013"
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,"04 27, 2014"
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,"03 15, 2014"
5,A27UF1MSF3DB2,0000031887,"C-Lo ""Cynthia""","[0, 0]",I received this today and I'm not a fan of it ...,4.0,It's ok,1396224000,"03 31, 2014"
6,A16GFPNVF4Y816,0000031887,design maven,"[0, 0]",Bought this as a backup to the regular ballet ...,5.0,Great for dress-up and for ballet practice,1399075200,"05 3, 2014"
7,A2M2APVYIB2U6K,0000031887,Jamie P.,"[0, 0]",Great tutu for a great price. It isn't a &#34;...,5.0,Great value,1356220800,"12 23, 2012"
8,A1NJ71X3YPQNQ9,0000031887,JBerger,"[0, 0]","My daughter liked this, and it with her costum...",4.0,Good,1384041600,"11 10, 2013"
9,A3EERSWHAI6SO,0000031887,"Jeffrey Hollingshead ""Jillian hollingshead""","[7, 8]",For what I paid for two tutus is unbeatable an...,5.0,WOW !! ..is all I have to say!,1349568000,"10 7, 2012"


In [5]:
#Check size of dataset 
df.shape

(278677, 9)

In [26]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [6]:
df[df['asin'] == '1608299953']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
29,A1F7YU6O5RU432,1608299953,Angela Streiff,"[0, 0]",Since learning a language from software is not...,5.0,"Suggest getting this 1 level at a time, so you...",1365465600,"04 9, 2013"
30,A3INPLAFCMRI3I,1608299953,Cheryl,"[201, 205]","The product itself is five stars, however the ...",4.0,4 stars BUT beware...,1322956800,"12 4, 2011"
31,A3NHUQ33CFH3VM,1608299953,Citizen John,"[5, 6]",The first question was which computers to inst...,4.0,Intense way to learn,1285027200,"09 21, 2010"
32,A253TILLU81VZK,1608299953,D. Connelly,"[11, 13]","Okay, I admit it. I'm one of the millions of A...",5.0,Helped me actually get an A in a foreign langu...,1308182400,"06 16, 2011"
33,A28QH3KX709FFQ,1608299953,"Don Quixote ""Don Q""","[24, 31]",I bought this product a little over a year ago...,1.0,DO NOT BUY IF YOU EVER WANT TO RESELL IT.,1348185600,"09 21, 2012"
34,A1R377IPZOKLMM,1608299953,"Gina B. ""It's a dry heat!""","[0, 3]",I really enjoy the Rosetta Stone software; it'...,5.0,Rosetta Stone software is fun to use,1284768000,"09 18, 2010"
35,A3U6J0DLLDEWM2,1608299953,Jason Stokes,"[62, 66]","I've never had french, learned french, or know...",5.0,Great way to obtain functional conversation in...,1287446400,"10 19, 2010"
36,A1ZU55TM45Y2R8,1608299953,J. Haggard,"[31, 34]",La Femme Boit...... I think that means the gir...,4.0,"Great software, not so fun to install, and hea...",1282694400,"08 25, 2010"
37,A1NIGUK4M2ST8,1608299953,mk,"[7, 9]",The Rosetta Stone process gets you involve thr...,4.0,"A Neat System of Learning, A BIt Pricey",1305676800,"05 18, 2011"
38,AQY5XBYSENNZQ,1608299953,Quickbeam,"[5, 6]",I have had this product for a while to evaluat...,4.0,"Really nice refresher, unable to get microphon...",1305763200,"05 19, 2011"


In [7]:
#Check how many unique items are in the data 
len(df['asin'].unique())

23033

In [12]:
#Created a non-repeating list of product ID #s with the intent of looping through them in the url to scrape. 
asin_set = set(df['asin'])
asin_set

{'B0083KJ2LW',
 'B004Z01W12',
 'B0047EXQ24',
 'B0089PROIO',
 'B00C5TOD4I',
 'B004I5BQBK',
 'B00GAO3YWG',
 'B00722CA16',
 'B003VFVSHU',
 'B003B2QJFO',
 'B00E4LD6B6',
 'B0028RZX9Q',
 'B00F4D105W',
 'B005FP21G6',
 'B002E3DDNM',
 'B00E1PAPTQ',
 'B008TKIT3S',
 'B00FPYZD1C',
 'B002NU6F2C',
 'B00AVPHH4Q',
 'B00B1ZM0C4',
 'B002ERJFTE',
 'B0009MZW34',
 'B008KH2H60',
 'B009RIZDSK',
 'B001941TTC',
 'B0040CBROM',
 'B001TI5H0A',
 'B008TP8MOE',
 'B0083LNDH0',
 'B00A2L5N0E',
 'B001U88BC0',
 'B000XKD298',
 'B009WOZSWA',
 'B001T0IM5U',
 'B00024QZX0',
 'B002XKHQDY',
 'B00B7PC622',
 'B00586BQ9U',
 'B0045EY918',
 'B00BOJBODI',
 'B004G93EJA',
 'B0047EXQZQ',
 'B008I18O42',
 'B00AYXF5HQ',
 'B00AY8M3IU',
 'B005NYO30Q',
 'B003VOYD2S',
 'B00EK94N1O',
 'B000NZSW84',
 'B00816S6EW',
 'B005CFVU9S',
 'B005OKDO22',
 'B009CG871C',
 'B005RSSXLS',
 'B003U92XXA',
 'B00AMZOSSS',
 'B00BJH6FD4',
 'B00BH4HOW0',
 'B0013JG4S4',
 'B004BJ1ZB4',
 'B00685IC9M',
 'B003TON3Q2',
 'B004CX0OQ6',
 'B00857TG0K',
 'B005E0CSVK',
 'B00BXJ29

In [13]:
#Check how many unique users are in the data. 
len(df['reviewerID'].unique())

39387

EDA

In [28]:
#Check columns types 
df.dtypes

reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
dtype: object

In [39]:
#Convert universal time to date
df['clean_time'] = pd.to_datetime(df['unixReviewTime'], unit='s')
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,clean_time
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,"02 12, 2011",2011-02-12
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,"01 19, 2013",2013-01-19
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,"01 4, 2013",2013-01-04
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,"04 27, 2014",2014-04-27
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,"03 15, 2014",2014-03-15


In [41]:
#Drop unuseful date and time columns 
df.drop(columns=['reviewTime', 'unixReviewTime'], inplace=True)

In [15]:
#Create a pivot table with the product ID as the rows, the userID as the columns and the overall rating as the values.
pivot = df.pivot_table(index='asin',
              columns='reviewerID',
              values='overall')

In [16]:
pivot.head()

reviewerID,A001114613O3F18Q5NVR6,A00146182PNM90WNNAZ5Q,A00165422B2GAUE3EL6Z0,A00338282E99B8OR2JYTZ,A00354001GE099Q1FL0TU,A00404823GU1Q517KP4Z8,A00475741WVLQL5CVWK2Z,A0055084JB2YQW2IDOSQ,A00635603LUUJQPQWSJW1,A007227915NZZC7B6P030,...,AZZ2HXC308RW1,AZZ9BV67BM0V0,AZZHJIL9S6XZ5,AZZHZZMH3U1VB,AZZKXNSXR6KBE,AZZMQ85DPFEG3,AZZNK89PXD006,AZZT1ERHBSNQ8,AZZTOUKVTUMVM,AZZYW4YOE1B6E
asin,,,,,,,,,,,,,,,,,,,,,
0000031887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0123456479,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1608299953,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1617160377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00001W0KA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Shape matches the num of unique users in the columns and num of unique items in the rows 
pivot.shape

(23033, 39387)

In [19]:
#Convert to sparse matrix to make smaller file size 
pivot_sparse = sparse.csr_matrix(pivot.fillna(0))

In [20]:
print(pivot_sparse[:5])

  (0, 1824)	5.0
  (0, 5623)	5.0
  (0, 5862)	5.0
  (0, 6526)	5.0
  (0, 6712)	4.0
  (0, 7923)	5.0
  (0, 12214)	3.0
  (0, 12672)	4.0
  (0, 12790)	5.0
  (0, 13319)	5.0
  (0, 15118)	5.0
  (0, 16827)	5.0
  (0, 18008)	5.0
  (0, 18835)	5.0
  (0, 20187)	1.0
  (0, 22176)	5.0
  (0, 23539)	5.0
  (0, 25109)	5.0
  (0, 25705)	5.0
  (0, 31651)	5.0
  (0, 34621)	5.0
  (0, 34843)	4.0
  (0, 38603)	5.0
  (1, 5496)	3.0
  (1, 9345)	5.0
  :	:
  (2, 36879)	4.0
  (3, 2497)	4.0
  (3, 10528)	5.0
  (3, 10727)	5.0
  (3, 11951)	5.0
  (3, 13803)	5.0
  (3, 16042)	4.0
  (3, 19585)	4.0
  (3, 21346)	5.0
  (3, 22995)	5.0
  (3, 26140)	4.0
  (3, 27419)	5.0
  (3, 29666)	4.0
  (3, 35757)	4.0
  (3, 36104)	5.0
  (3, 38888)	5.0
  (3, 39186)	5.0
  (4, 112)	4.0
  (4, 601)	5.0
  (4, 4484)	4.0
  (4, 11432)	5.0
  (4, 19590)	4.0
  (4, 19631)	5.0
  (4, 23062)	5.0
  (4, 31246)	5.0


In [21]:
# Check similiarity of every item to every other item. 
recommender = pairwise_distances(pivot_sparse, metric='cosine')

In [22]:
#Checking to make sure it's a square. 
recommender.shape

(23033, 23033)

In [23]:
#Convert to DataFrame 
recommender_df = pd.DataFrame(recommender, index=pivot.index, columns=pivot.index)

In [24]:
recommender_df.head()

asin,0000031887,0123456479,1608299953,1617160377,B00001W0KA,B00001WRHJ,B00004SR8W,B00004SR8Z,B00004SR9P,B00004U1J2,...,B00K0BPFY0,B00K551QR6,B00K5T4NHC,B00K8J06CK,B00KA2X4QK,B00KA602SY,B00KCWMG5S,B00KF9180W,B00KGCLROK,B00KKXCJQU
asin,,,,,,,,,,,,,,,,,,,,,
0000031887,0.0,1.0,1.0000,1.0000,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0
0123456479,1.0,0.0,1.0000,1.0000,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0
1608299953,1.0,1.0,0.0000,0.9328,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0
1617160377,1.0,1.0,0.9328,0.0000,1.0,1.0,1.0,1.0,1.0,1.000000,...,1.0,1.0,0.895685,1.0,1.0,1.0,1.0,0.911762,0.868209,1.0
B00001W0KA,1.0,1.0,1.0000,1.0000,0.0,1.0,1.0,1.0,1.0,0.574821,...,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.000000,1.000000,1.0


In [25]:
recommender_df['B00001W0KA'].sort_values()[1:11]

asin
B00004U1J2    0.574821
B005JJ2PK0    0.802905
B00551RXJK    0.810872
B006PHRPG4    0.810872
B0072QV4V4    0.814727
B0009ETG02    0.815289
B00EF0XQAW    0.826715
B004JV74VY    0.829995
B004M47EB8    0.847943
B00DNKFAT0    0.853357
Name: B00001W0KA, dtype: float64

In [27]:
# A rough attempt at building a recommender and I see that it is important that I identify the product names 
# to match with the asin.  